In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
engine = create_engine("sqlite:///:memory:", echo=True)
Session = sessionmaker(bind=engine)
session = Session()

In [3]:
from datetime import datetime
from sqlalchemy import (
    Table,
    Column,
    Integer,
    Numeric,
    String,
    DateTime,
    Boolean,
    ForeignKey,
)
from sqlalchemy.orm import relationship, backref, declarative_base

In [4]:
Base = declarative_base()


class Cookie(Base):
    __tablename__ = "cookies"

    cookie_id = Column(Integer, primary_key=True)
    cookie_name = Column(String(50), index=True)
    cookie_recipe_url = Column(String(255))
    cookie_sku = Column(String(55))
    quantity = Column(Integer)
    unit_cost = Column(Numeric(12, 2))

    def __repr__(self):
        return f"Cookie(cookie_name={self.cookie_name},cookie_recipe_url={self.cookie_recipe_url},cookie_skue={self.cookie_sku},quantity={self.quantity},unit_cost={self.unit_cost})"

In [5]:
class User(Base):
    __tablename__ = "users"

    user_id = Column(Integer, primary_key=True)
    username = Column(String(15), nullable=False, unique=True)
    email_address = Column(String(255), nullable=False)
    phone = Column(String(20), nullable=False)
    password = Column(String(25), nullable=False)
    create_on = Column(DateTime(), default=datetime.now)
    update_on = Column(DateTime(), default=datetime.now, onupdate=datetime.now)

    def __repr__(self):
        return f"User(username={self.username},email_address={self.email_address},phone={self.phone},password={self.password})"


class Order(Base):
    __tablename__ = "orders"

    order_id = Column(Integer(), primary_key=True)
    user_id = Column(Integer(), ForeignKey("users.user_id"))
    user = relationship("User", backref=backref("orders", order_by=order_id))

    def __repr__(self):
        return "Order(user_id={self.user_id},shipped={self.shipped})"


class Lineitems(Base):
    __tablename__ = "line_items"
    line_item_id = Column(Integer(), primary_key=True)
    order_id = Column(Integer(), ForeignKey("orders.order_id"))
    cookie_id = Column(Integer(), ForeignKey("cookies.cookie_id"))
    quantity = Column(Integer())
    extended_cost = Column(Numeric(12, 2))
    order = relationship("Order", backref=backref("line_items", order_by=line_item_id))
    cookie = relationship("Cookie", uselist=False, order_by=cookie_id)

    def __repr__(self):
        return "Lineitems(order_id={self.order_id},cookie_id={self.cookie_id},quantity={self.quantity},extended_cost={self.extened_cost})"


Base.metadata.create_all(engine)

2025-05-22 10:48:00,480 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-22 10:48:00,482 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("cookies")
2025-05-22 10:48:00,483 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-22 10:48:00,485 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("cookies")
2025-05-22 10:48:00,486 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-22 10:48:00,487 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-05-22 10:48:00,488 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-22 10:48:00,489 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2025-05-22 10:48:00,490 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-22 10:48:00,493 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")
2025-05-22 10:48:00,494 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-22 10:48:00,497 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("orders")
2025-05-22 10:48:00,498 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-

In [6]:
cc = Cookie(
    cookie_name="chocolate chip",
    cookie_recipe_url="http://some.aweso.me/cookie/recippe.html",
    cookie_sku="cc01",
    quantity=12,
    unit_cost=0.5,
)
session.add(cc)
session.commit()

2025-05-22 10:51:43,404 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-22 10:51:43,407 INFO sqlalchemy.engine.Engine INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (?, ?, ?, ?, ?)
2025-05-22 10:51:43,407 INFO sqlalchemy.engine.Engine [generated in 0.00090s] ('chocolate chip', 'http://some.aweso.me/cookie/recippe.html', 'cc01', 12, 0.5)
2025-05-22 10:51:43,410 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
cc.cookie_id

2025-05-22 10:52:59,509 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-22 10:52:59,513 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies 
WHERE cookies.cookie_id = ?
2025-05-22 10:52:59,514 INFO sqlalchemy.engine.Engine [generated in 0.00106s] (1,)


1

In [8]:
dcc = Cookie(
    cookie_name="dark chocolate chip",
    cookie_recipe_url="http://some.aweso.me/cookie/recipe_dark.html",
    cookie_sku="CC02",
    quantity=1,
    unit_cost=0.75,
)
mol = Cookie(
    cookie_name="molasses",
    cookie_recipe_url="http://some.aweso.me/cookie/recipe_molasses.html",
    cookie_sku="MOL01",
    quantity=1,
    unit_cost=0.80,
)
session.add(dcc)
session.add(mol)
session.flush()
print(dcc.cookie_id)
print(mol.cookie_id)

2025-05-22 10:55:24,119 INFO sqlalchemy.engine.Engine INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (?, ?, ?, ?, ?) RETURNING cookie_id
2025-05-22 10:55:24,121 INFO sqlalchemy.engine.Engine [generated in 0.00021s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('dark chocolate chip', 'http://some.aweso.me/cookie/recipe_dark.html', 'CC02', 1, 0.75)
2025-05-22 10:55:24,122 INFO sqlalchemy.engine.Engine INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (?, ?, ?, ?, ?) RETURNING cookie_id
2025-05-22 10:55:24,123 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('molasses', 'http://some.aweso.me/cookie/recipe_molasses.html', 'MOL01', 1, 0.8)
2
3


In [9]:
c1 = Cookie(
    cookie_name="peanut butter",
    cookie_recipe_url="http://some.aweso.me/cookie/peanut.html",
    cookie_sku="PB01",
    quantity=24,
    unit_cost=0.25,
)
c2 = Cookie(
    cookie_name="oatmeal raisin",
    cookie_recipe_url="http://some.okay.me/cookie/raisin.html",
    cookie_sku="EWW01",
    quantity=100,
    unit_cost=1.00,
)
session.bulk_save_objects([c1, c2])
session.commit()
print(c1.cookie_id)

2025-05-22 10:58:06,805 INFO sqlalchemy.engine.Engine INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (?, ?, ?, ?, ?)
2025-05-22 10:58:06,807 INFO sqlalchemy.engine.Engine [generated in 0.00189s] [('peanut butter', 'http://some.aweso.me/cookie/peanut.html', 'PB01', 24, 0.25), ('oatmeal raisin', 'http://some.okay.me/cookie/raisin.html', 'EWW01', 100, 1.0)]
2025-05-22 10:58:06,808 INFO sqlalchemy.engine.Engine COMMIT
None


In [10]:
from sqlalchemy import text

with engine.connect() as cnn:
    result = cnn.execute(text("select * from cookies"))
    print(result.fetchall())

2025-05-22 11:01:09,228 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-22 11:01:09,229 INFO sqlalchemy.engine.Engine select * from cookies
2025-05-22 11:01:09,230 INFO sqlalchemy.engine.Engine [generated in 0.00265s] ()
[(1, 'chocolate chip', 'http://some.aweso.me/cookie/recippe.html', 'cc01', 12, 0.5), (2, 'dark chocolate chip', 'http://some.aweso.me/cookie/recipe_dark.html', 'CC02', 1, 0.75), (3, 'molasses', 'http://some.aweso.me/cookie/recipe_molasses.html', 'MOL01', 1, 0.8), (4, 'peanut butter', 'http://some.aweso.me/cookie/peanut.html', 'PB01', 24, 0.25), (5, 'oatmeal raisin', 'http://some.okay.me/cookie/raisin.html', 'EWW01', 100, 1)]
2025-05-22 11:01:09,233 INFO sqlalchemy.engine.Engine ROLLBACK


In [11]:
ccs = session.query(Cookie).all()
print(ccs)

2025-05-22 11:02:41,899 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-22 11:02:41,902 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies
2025-05-22 11:02:41,903 INFO sqlalchemy.engine.Engine [generated in 0.00089s] ()
[Cookie(cookie_name=chocolate chip,cookie_recipe_url=http://some.aweso.me/cookie/recippe.html,cookie_skue=cc01,quantity=12,unit_cost=0.50), Cookie(cookie_name=dark chocolate chip,cookie_recipe_url=http://some.aweso.me/cookie/recipe_dark.html,cookie_skue=CC02,quantity=1,unit_cost=0.75), Cookie(cookie_name=molasses,cookie_recipe_url=http://some.aweso.me/cookie/recipe_molasses.html,cookie_skue=MOL01,quantity=1,unit_cost=0.80), Cookie(cookie_name=peanut butter,cookie_recipe_url=http://some.aweso.me/cookie/peanut.

In [14]:
from pprint import pprint

pprint(ccs)

[Cookie(cookie_name=chocolate chip,cookie_recipe_url=http://some.aweso.me/cookie/recippe.html,cookie_skue=cc01,quantity=12,unit_cost=0.50),
 Cookie(cookie_name=dark chocolate chip,cookie_recipe_url=http://some.aweso.me/cookie/recipe_dark.html,cookie_skue=CC02,quantity=1,unit_cost=0.75),
 Cookie(cookie_name=molasses,cookie_recipe_url=http://some.aweso.me/cookie/recipe_molasses.html,cookie_skue=MOL01,quantity=1,unit_cost=0.80),
 Cookie(cookie_name=peanut butter,cookie_recipe_url=http://some.aweso.me/cookie/peanut.html,cookie_skue=PB01,quantity=24,unit_cost=0.25),
 Cookie(cookie_name=oatmeal raisin,cookie_recipe_url=http://some.okay.me/cookie/raisin.html,cookie_skue=EWW01,quantity=100,unit_cost=1.00)]


In [15]:
for cookie in session.query(Cookie):
    print(cookie)

2025-05-22 11:04:15,389 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies
2025-05-22 11:04:15,391 INFO sqlalchemy.engine.Engine [cached since 91.22s ago] ()
Cookie(cookie_name=chocolate chip,cookie_recipe_url=http://some.aweso.me/cookie/recippe.html,cookie_skue=cc01,quantity=12,unit_cost=0.50)
Cookie(cookie_name=dark chocolate chip,cookie_recipe_url=http://some.aweso.me/cookie/recipe_dark.html,cookie_skue=CC02,quantity=1,unit_cost=0.75)
Cookie(cookie_name=molasses,cookie_recipe_url=http://some.aweso.me/cookie/recipe_molasses.html,cookie_skue=MOL01,quantity=1,unit_cost=0.80)
Cookie(cookie_name=peanut butter,cookie_recipe_url=http://some.aweso.me/cookie/peanut.html,cookie_skue=PB01,quantity=24,unit_cost=0.25)
Cookie(cookie_name=oatm

In [16]:
print(session.query(Cookie.cookie_name, Cookie.quantity).first())

2025-05-22 11:35:13,590 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_name AS cookies_cookie_name, cookies.quantity AS cookies_quantity 
FROM cookies
 LIMIT ? OFFSET ?
2025-05-22 11:35:13,592 INFO sqlalchemy.engine.Engine [generated in 0.00209s] (1, 0)
('chocolate chip', 12)


In [18]:
for cookie in session.query(Cookie).order_by(Cookie.quantity):
    print(f"{cookie.quantity:3} - {cookie.cookie_name}")

2025-05-22 11:37:33,855 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies ORDER BY cookies.quantity
2025-05-22 11:37:33,857 INFO sqlalchemy.engine.Engine [generated in 0.00246s] ()
  1 - dark chocolate chip
  1 - molasses
 12 - chocolate chip
 24 - peanut butter
100 - oatmeal raisin


In [23]:
from sqlalchemy import desc

for cookie in session.query(Cookie).order_by(desc(Cookie.quantity)):
    print(f"{cookie.quantity:<15}{cookie.cookie_name}")

2025-05-22 11:39:58,073 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies ORDER BY cookies.quantity DESC
2025-05-22 11:39:58,075 INFO sqlalchemy.engine.Engine [cached since 31.66s ago] ()
100            oatmeal raisin
24             peanut butter
12             chocolate chip
1              dark chocolate chip
1              molasses


In [28]:
query = session.query(Cookie).order_by(Cookie.quantity)[:2]
print([(result.cookie_name, result.quantity) for result in query])

2025-05-22 11:43:06,889 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies ORDER BY cookies.quantity
 LIMIT ? OFFSET ?
2025-05-22 11:43:06,891 INFO sqlalchemy.engine.Engine [cached since 59.81s ago] (2, 0)
[('dark chocolate chip', 1), ('molasses', 1)]


In [35]:
query = session.query(Cookie).order_by(Cookie.quantity).limit(2)
print([(result.cookie_name, result.quantity) for result in query])

2025-05-22 11:46:40,805 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies ORDER BY cookies.quantity
 LIMIT ? OFFSET ?
2025-05-22 11:46:40,806 INFO sqlalchemy.engine.Engine [cached since 268.7s ago] (2, 0)
[('dark chocolate chip', 1), ('molasses', 1)]


In [39]:
from sqlalchemy import func

inv_count = session.query(func.sum(Cookie.quantity))
print(inv_count.all())

2025-05-22 11:48:51,107 INFO sqlalchemy.engine.Engine SELECT sum(cookies.quantity) AS sum_1 
FROM cookies
2025-05-22 11:48:51,108 INFO sqlalchemy.engine.Engine [cached since 58.76s ago] ()
[(138,)]


In [41]:
inv_count = session.query(func.sum(Cookie.quantity))
print(inv_count.count())

2025-05-22 11:49:25,173 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT sum(cookies.quantity) AS sum_1 
FROM cookies) AS anon_1
2025-05-22 11:49:25,175 INFO sqlalchemy.engine.Engine [generated in 0.00238s] ()
1


In [43]:
cnt = session.query(func.count(Cookie.cookie_name)).scalar()
print(cnt)

2025-05-22 11:50:33,992 INFO sqlalchemy.engine.Engine SELECT count(cookies.cookie_name) AS count_1 
FROM cookies
2025-05-22 11:50:33,993 INFO sqlalchemy.engine.Engine [generated in 0.00161s] ()
5


In [44]:
cnt = session.query(func.count(Cookie.cookie_name).label("inv_count")).first()
print(cnt._key_to_index)

2025-05-22 11:52:11,158 INFO sqlalchemy.engine.Engine SELECT count(cookies.cookie_name) AS inv_count 
FROM cookies
 LIMIT ? OFFSET ?
2025-05-22 11:52:11,160 INFO sqlalchemy.engine.Engine [generated in 0.00197s] (1, 0)
{'inv_count': 0, <sqlalchemy.sql.elements.Label at 0x7f657932c830; inv_count>: 0}


In [46]:
cnt.inv_count

5

In [48]:
rec = session.query(Cookie).filter(Cookie.cookie_name == "chocolate chip").first()
print(rec)

2025-05-22 11:54:00,632 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies 
WHERE cookies.cookie_name = ?
 LIMIT ? OFFSET ?
2025-05-22 11:54:00,634 INFO sqlalchemy.engine.Engine [generated in 0.00190s] ('chocolate chip', 1, 0)
Cookie(cookie_name=chocolate chip,cookie_recipe_url=http://some.aweso.me/cookie/recippe.html,cookie_skue=cc01,quantity=12,unit_cost=0.50)


In [49]:
rec = session.query(Cookie).filter_by(cookie_name="chocolate chip").first()
print(rec)

2025-05-22 14:09:45,675 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies 
WHERE cookies.cookie_name = ?
 LIMIT ? OFFSET ?
2025-05-22 14:09:45,677 INFO sqlalchemy.engine.Engine [cached since 7920s ago] ('chocolate chip', 1, 0)
Cookie(cookie_name=chocolate chip,cookie_recipe_url=http://some.aweso.me/cookie/recippe.html,cookie_skue=cc01,quantity=12,unit_cost=0.50)


In [50]:
query = session.query(Cookie).filter(Cookie.cookie_name.like("%chocolate%"))
for row in query:
    print(row.cookie_name)

2025-05-22 14:11:11,809 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies 
WHERE cookies.cookie_name LIKE ?
2025-05-22 14:11:11,811 INFO sqlalchemy.engine.Engine [generated in 0.00220s] ('%chocolate%',)
chocolate chip
dark chocolate chip
